In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"pratikbarua","key":"cdfa4671b4f8030e00497e5760b11046"}'}

In [2]:
!ls -lha kaggle.json

-rw-r--r-- 1 root root 67 Jun  7 04:05 kaggle.json


In [0]:
!pip install -q kaggle

In [0]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d mushfiqurrobin/world-cup-results

  0% 0.00/13.0k [00:00<?, ?B/s]
100% 13.0k/13.0k [00:00<00:00, 22.9MB/s]


In [0]:
import tensorflow as tf
import os
import zipfile
from os import path, getcwd, chdir

path = "/content/world-cup-results.zip"
zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

In [0]:
import pandas as pd
import matplotlib as plt
from matplotlib import pyplot
pyplot.style.use('ggplot')

In [9]:
#file path
path="/tmp/world_cup_results.csv"
#importin csv file
df=pd.read_csv(path)
#printing csv file
df.head()

,Year,Date,Time,Round,Stadium,City,HomeTeam,HomeGoals,AwayGoals,AwayTeam,Observation
0,1930,13-Jul-30,15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,
1,1930,13-Jul-30,15:00,Group 4,Parque Central,Montevideo,USA,3,0,Belgium,
2,1930,14-Jul-30,12:45,Group 2,Parque Central,Montevideo,Yugoslavia,2,1,Brazil,
3,1930,14-Jul-30,14:50,Group 3,Pocitos,Montevideo,Romania,3,1,Peru,
4,1930,15-Jul-30,16:00,Group 1,Parque Central,Montevideo,Argentina,1,0,France,


In [31]:
df.shape

(852, 11)

In [32]:
#let's deal with the duplicates
#Notice that from a shape of (852cols, 11rows) we now arrive at (836, 11). There was duplicates
df = df.drop_duplicates()
df.shape



(836, 11)

In [0]:
#Total goals column
df['TotalGoals'] = df['HomeGoals'] + df['AwayGoals']

In [0]:
df['month'] = df['Date'].apply(lambda x: x.split('-')[1])
df['day'] = pd.to_datetime(df['Date']).dt.day_name()

In [35]:
#Let's see what our df looks like now by peeping the head
df.head(2)

,Year,Date,Time,Round,Stadium,City,HomeTeam,HomeGoals,AwayGoals,AwayTeam,Observation,TotalGoals,month,day
0,1930,13-Jul-30,15:00,Group 1,Pocitos,Montevideo,France,4,1,Mexico,,5,Jul,Saturday
1,1930,13-Jul-30,15:00,Group 4,Parque Central,Montevideo,USA,3,0,Belgium,,3,Jul,Saturday


In [36]:
df = df[['Year', 'month', 'day', 'Time', 'Round', 'HomeTeam', 'HomeGoals', 'AwayTeam', 'AwayGoals', 'TotalGoals']]
df.head(2)

,Year,month,day,Time,Round,HomeTeam,HomeGoals,AwayTeam,AwayGoals,TotalGoals
0,1930,Jul,Saturday,15:00,Group 1,France,4,Mexico,1,5
1,1930,Jul,Saturday,15:00,Group 4,USA,3,Belgium,0,3


**How many matches were played each world cup year from 1930.**

In [43]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
#A value_count on the Year column nicely delivers this
#To plot this effortlessly with plotly, we will convert the result to a fresh dataframe
#Notice how nicely plotly highlights the expected years world cups were not played
matches_per_year = df.Year.value_counts() #a series to hold our values
all_games = pd.DataFrame(matches_per_year) #make series into a dataframe
all_games.reset_index(inplace=True) #reset it's index inplace
all_games.columns = ['Year', 'Matches'] #rename the columns as needed
fig = px.bar(all_games, x='Year', y='Matches', text='Matches', color='Matches', height=500,
labels={'Matches':'Matches Played', 'Year':'World Cup Year'},
title="Total Matches Played Each World Cup Year")
fig.update_traces(texttemplate='%{text}', textposition='outside')
fig.update_layout()
fig.update_xaxes(
tickangle=45, tickfont=dict(family='Arial', color='blue', size=14),
tickvals=[line for line in all_games.Year])
fig.show()

**Total goals scored for each tournament year.**
1.   Group initial dataframe by year summing total goals
2.   Convert result to a dataframe and drop a plot on the result



In [46]:
all_goals = df.groupby(['Year']).TotalGoals.sum()
all_goals_df = pd.DataFrame(all_goals)
all_goals_df.reset_index(inplace=True) #reset it's index inplace
all_goals_df.columns = ['Year', 'Goals'] #rename the columns as needed
fig = px.bar(all_goals_df, x='Year', y='Goals', text='Goals', color='Goals', height=450,
labels={'Goals':'Goals Scored', 'Year':'World Cup Year'},
title="Total Goals Scored Each World Cup Year")
fig.update_traces(texttemplate='%{text}', textposition='inside')
fig.update_layout()
fig.update_xaxes(
tickangle=45, tickfont=dict(family='Arial', color='blue', size=14),
tickvals=[line for line in all_goals_df.Year])
fig.show()

**All teams who have reached finals and how many times.**

In [47]:
#Take a piece of the df corresponding to all 'Final' in the 'Round' column
all_finals = df[df['Round'] == 'Final']
all_finals.head()

,Year,month,day,Time,Round,HomeTeam,HomeGoals,AwayTeam,AwayGoals,TotalGoals
17,1930,Jul,Tuesday,14:15,Final,Uruguay,4,Argentina,2,6
34,1934,Jun,Saturday,17:30,Final,Italy,2,Czechoslovakia,1,3
52,1938,Jun,Saturday,17:00,Final,Italy,4,Hungary,2,6
100,1954,Jul,Saturday,17:00,Final,Germany FR,3,Hungary,2,5
135,1958,Jun,Saturday,15:00,Final,Brazil,5,Sweden,2,7


In [50]:
#Let's make a list of all teams who reach this stage.
#This will be a list of all featuring HomeTeams and AwayTeams
#A simple concatenation of a python list of both will do
teams = [line for line in all_finals.HomeTeam] + [line for line in all_finals.AwayTeam]
#Peep a sample
teams

['Uruguay',
 'Italy',
 'Italy',
 'Germany FR',
 'Brazil',
 'Brazil',
 'England',
 'Brazil',
 'Netherlands',
 'Argentina',
 'Italy',
 'Argentina',
 'Germany FR',
 'Brazil',
 'Brazil',
 'Germany',
 'Italy',
 'Netherlands',
 'Germany',
 'Argentina',
 'Czechoslovakia',
 'Hungary',
 'Hungary',
 'Sweden',
 'Czechoslovakia',
 'Germany FR',
 'Italy',
 'Germany FR',
 'Netherlands',
 'Germany FR',
 'Germany FR',
 'Argentina',
 'Italy',
 'France',
 'Brazil',
 'France',
 'Spain',
 'Argentina']

In [51]:
#To count the frequency that I am going to eventually plot I prefer to use a dataframe. It's seamless
#So I will make a dataframe from the list 'teams' and take a drop a value_counts(). Neat, yea?
all_finals_df = pd.DataFrame(columns=['Teams'], data = teams)
#peep the head()
all_finals_df.head(2)

,Teams
0,Uruguay
1,Italy


In [52]:
#To demonstrate this value_counts() counting, see the result before we plot
#PS: I am choosing to leave Germany and Germany FR as different entities
all_finals_df.Teams.value_counts()

Brazil            6
Germany FR        6
Italy             6
Argentina         5
Netherlands       3
Czechoslovakia    2
France            2
Hungary           2
Germany           2
Sweden            1
Spain             1
England           1
Uruguay           1
Name: Teams, dtype: int64

In [54]:
#Make a dataframe from counting values in all_finals_df
finals_teams_ranked = all_finals_df.Teams.value_counts()
finals_teams_ranked_df = pd.DataFrame(finals_teams_ranked)
finals_teams_ranked_df.reset_index(inplace=True)
finals_teams_ranked_df.columns = ['Teams', 'Frequency'] #rename the columns as needed
fig = px.bar(finals_teams_ranked_df, x='Teams', y='Frequency', color='Frequency', height=450,
labels={'Teams':'Teams in the Finals'},
title="All Teams Who Have Reached Finals and Frequency")
fig.update_layout()
fig.show()

**All teams who have reached semis and how many times.**

In [56]:
#Take a piece of the df corresponding to all 'Semi-finals' in the 'Round' column
all_semi_finals = df[df['Round'] == 'Semi-finals']
all_semi_finals.head()

,Year,month,day,Time,Round,HomeTeam,HomeGoals,AwayTeam,AwayGoals,TotalGoals
15,1930,Jul,Friday,14:45,Semi-finals,Argentina,6,USA,1,7
16,1930,Jul,Saturday,14:45,Semi-finals,Uruguay,6,Yugoslavia,1,7
31,1934,Jun,Saturday,16:30,Semi-finals,Italy,1,Austria,0,1
32,1934,Jun,Saturday,16:30,Semi-finals,Czechoslovakia,3,Germany,1,4
49,1938,Jun,Wednesday,18:00,Semi-finals,Hungary,5,Sweden,1,6


In [0]:
#Make a list of all teams invovled Home and Away
teams = [line for line in all_semi_finals.HomeTeam] + [line for line in all_semi_finals.AwayTeam]

In [58]:
#Make a df of teams
all_semi_finals_df = pd.DataFrame(columns=['Teams'], data = teams)
#peep the head()
all_semi_finals_df.head(2)

,Teams
0,Argentina
1,Uruguay


In [59]:
#Make a dataframe from counting values in all_semi_finals
#See 'Finals' cell above for explanation as the steps are identical. We are avoiding functions for practise
semifinals_teams_ranked = all_semi_finals_df.Teams.value_counts()
semifinals_teams_ranked_df = pd.DataFrame(semifinals_teams_ranked)
semifinals_teams_ranked_df.reset_index(inplace=True)
semifinals_teams_ranked_df.columns = ['Teams', 'Frequency']
fig = px.bar(finals_teams_ranked_df, x='Teams', y='Frequency', color='Frequency', height=450,
labels={'Teams':'Teams in the Finals'},
title="All Teams Who Have Reached Semi-Final and Frequency")
fig.update_layout()
fig.show()

**How many goals and average goals scored in all semi-finals.**

In [63]:
#Continue with our dataframe holding only semi-final matches
#Let's take a sum and mean of 'TotalGoals'
semi_goals_sum = all_semi_finals.TotalGoals.sum()
semi_goals_ave = all_semi_finals.TotalGoals.mean()
print(f"{semi_goals_sum} goals were scored in all Semi-Finals\nAn average of {semi_goals_ave:.2f} in every match.")

123 goals were scored in all Semi-Finals
An average of 3.62 in every match.


**How many goals and average goals scored in all quarter-finals.**

In [61]:
#Get a slice of the original df for all quarter-finals
all_qtrs = df[df['Round'] == 'Quarter-finals']
all_qtrs.head()

,Year,month,day,Time,Round,HomeTeam,HomeGoals,AwayTeam,AwayGoals,TotalGoals
26,1934,May,Wednesday,16:30,Quarter-finals,Czechoslovakia,3,Switzerland,2,5
27,1934,May,Wednesday,16:30,Quarter-finals,Germany,2,Sweden,1,3
28,1934,May,Wednesday,16:30,Quarter-finals,Italy,1,Spain,1,2
29,1934,May,Wednesday,16:30,Quarter-finals,Austria,2,Hungary,1,3
30,1934,Jun,Thursday,16:30,Quarter-finals,Italy,1,Spain,0,1


In [62]:
qtrs_goals_sum = all_qtrs.TotalGoals.sum()
qtrs_goals_ave = all_qtrs.TotalGoals.mean()
print(f"{qtrs_goals_sum} goals were scored in all Semi-Finals\nAn average of {qtrs_goals_ave:.2f} in every match.")

175 goals were scored in all Semi-Finals
An average of 2.82 in every match.


In [64]:
#All goals in finals
finals = df[df['Round'] == 'Final']['HomeGoals'].sum() + df[df['Round'] == 'Final']['AwayGoals'].sum()#[['HomeTeam',
print(f"{finals} goals in finals")

68 goals in finals


**How many goals and average number scored in all finals**

In [66]:
#Working with the piece of the original df holding finals
finals_goals_sum = all_finals.TotalGoals.sum()
finals_goals_ave = all_finals.TotalGoals.mean()
print(f"{finals_goals_sum} goals were scored in all Semi-Finals\nAn average of {finals_goals_ave:.2f} in every match")

68 goals were scored in all Semi-Finals
An average of 3.58 in every match


**How many matches were played outside quarter-finals and above.**

In [0]:
#Let's be creative here!
#First get a slice with no finals
df_less_finals = df[df['Round'] != 'Final']
#From there get a slies with no semi-finals and viola we are left with all matches neither finals or semis
df_less_finals_semis = df_less_finals[df_less_finals['Round'] != 'Semi-Finals']
#One more dropping qtrs. This is fun.
df_less_finals_semis_qtrs = df_less_finals_semis[df_less_finals_semis['Round'] != 'Quarter-finals']

In [68]:
#Did it work? Well, let's check!
'Final' in df_less_finals_semis_qtrs.Round.tolist() or 'Semi-Finals' in df_less_finals_semis_qtrs.Round.tolist()

False

In [69]:
#Just in case that was lady-luck, let's make sure other Rounds are there
'Round of 16' in df_less_finals_semis_qtrs.Round.tolist()

True

In [70]:
#Total matches in this slice of the dataframe is same number of rows. A number of move will show the number
d_rest0 = df_less_finals_semis_qtrs.shape[0]
d_rest1 = len(df_less_finals_semis_qtrs)
d_rest0 == d_rest1

True

In [71]:
print(f"There are {d_rest0} matches played outside Quater-finals and above")

There are 755 matches played outside Quater-finals and above


**The Kicker.**
Two new columns for each of the outcome of every match stating

1. outcome = D for Draw, A for AwayTeam Wins, H for HomeTeam wins.
2. Winner of each game: 'Draw' if no winner.






In [0]:
#I love python lists a lot as I know them in and out
#I will use a zip of four different columns from the datafarame to solve the kicker
# a python list of all four columns we are considering
AwayT_list = df['AwayTeam'].tolist()
HomeT_list = df['HomeTeam'].tolist()
AwayG_list = df['AwayGoals'].tolist()
HomeG_list = df['HomeGoals'].tolist()
#Two empty lists to hold our values for the two new columns
verdict, winner = [], []
#We zip the four lists created and step through them looking for the kicker condition, assigning values as we go
for at, ht, ag, hg in zip (AwayT_list, HomeT_list, AwayG_list, HomeG_list):
    if ag > hg:
       verdict.append('A')
       winner.append (at)
    elif hg > ag:
       verdict.append('H')
       winner.append(ht)
    elif hg == ag:
     if ag == 0:
      verdict.append('D')
      winner.append('Draw')
     else:
      verdict.append('A')
      winner.append(at)
#Finally write the two new columns to our dataframe
df['Verdict'] = verdict
df['Winner'] = winner

In [74]:
df.sample(10)

,Year,month,day,Time,Round,HomeTeam,HomeGoals,AwayTeam,AwayGoals,TotalGoals,Verdict,Winner
718,2010,Jun,Monday,20:30,Group F,Italy,1,Paraguay,1,2,A,Paraguay
709,2010,Jun,Friday,20:30,Group A,Uruguay,0,France,0,0,D,Draw
144,1962,Jun,Friday,15:00,Group 1,Yugoslavia,3,Uruguay,1,4,H,Yugoslavia
96,1954,Jun,Saturday,17:00,Quarter-finals,Hungary,4,Brazil,2,6,H,Hungary
478,1994,Jun,Wednesday,19:30,Group A,USA,2,Colombia,1,3,H,USA
785,2014,Jun,Monday,19:00,Group G,Ghana,1,USA,2,3,A,USA
255,1974,Jun,Sunday,16:00,Group 4,Poland,2,Italy,1,3,H,Poland
78,1954,Jun,Tuesday,18:00,Group 1,Yugoslavia,1,France,0,1,H,Yugoslavia
465,1994,Jun,Friday,15:00,Group C,Germany,1,Bolivia,0,1,H,Germany
729,2010,Jun,Friday,16:00,Group C,Slovenia,2,USA,2,4,A,USA


In [75]:
#Assemble all games played in Finals and Semi-finals
finals = df[df['Round'] == 'Final']
semis = df[df['Round']== 'Semi-finals']
#Conct both dataframes resetting the index
finals_semis = pd.concat([finals, semis]).reset_index(drop=True)
finals_semis.shape

(53, 12)

In [77]:
#Let's cherry-pick the columns we need
plot_df = finals_semis[['TotalGoals', 'Round', 'Year', 'month', 'day']]
plot_df

,TotalGoals,Round,Year,month,day
0,6,Final,1930,Jul,Tuesday
1,3,Final,1934,Jun,Saturday
2,6,Final,1938,Jun,Saturday
3,5,Final,1954,Jul,Saturday
4,7,Final,1958,Jun,Saturday
5,4,Final,1962,Jun,Saturday
6,6,Final,1966,Jul,Friday
7,5,Final,1970,Jun,Sunday
8,3,Final,1974,Jul,Sunday
9,4,Final,1978,Jun,Sunday


In [78]:
#The Plot
values2 = [68, 123, 36, 32, 65, 58, 20, 16, 6, 15, 5, 6, 25, 24, 5, 11, 11, 19, 8, 3, 7, 3, 7]
fig = go.Figure(go.Sunburst(
labels=[
"Final", "Semi-finals", "Jun", "Jul", 'Jun ', 'Jul ',
'Saturday', 'Sunday',
'Sunday ', 'Friday ', 'Saturday ', 'Tuesday ',
'Monday ', 'Saturday ', 'Tuesday ', 'Wednesday ',
'Friday ', 'Monday ', 'Sunday ', 'Saturday ', 'Thursday ', 'Tuesday ', 'Wednesday '
],
parents=[
"", "", "Final", "Final", 'Semi-finals', 'Semi-finals',
'Jun', 'Jun',
'Jul', 'Jul', 'Jul', 'Jul',
'Jun ', 'Jun ', 'Jun ', 'Jun ',
'Jul ', 'Jul ', 'Jul ', 'Jul ', 'Jul ', 'Jul ', 'Jul '
],
values=values2),
layout=go.Layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
)
fig.update_layout(margin=dict(t=0, l=0, r=0, b=0), title_text='Matches')
fig.data[0].marker=dict(colors=px.colors.sequential.Aggrnyl)
fig.show()